In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import gaussian
from scipy.ndimage import gaussian_filter
import os
import cv2

In [30]:
def extract_contours(mask):
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_image = np.zeros_like(mask)
    cv2.drawContours(contour_image, contours, -1, (255, 255, 255), 2)

    return contour_image

In [31]:
def generate_heatmaps(img_h, img_w, coords, dilation_size=10):
    num_coords = len(coords)
    heatmaps = np.zeros((num_coords, img_h, img_w))

    for i in range(num_coords):
        current_coord = coords[i]
        heatmaps[i, current_coord[0], current_coord[1]] = 1
    return heatmaps

In [32]:
def apply_gaussian_filter(heatmaps, var=20):
    gaussian_heatmaps = gaussian_filter(heatmaps, sigma=[0, var, var])
    return gaussian_heatmaps

In [33]:
def apply_contour_mask(gaussian_heatmaps, contour_image):
    gray_contour_image = cv2.cvtColor(contour_image, cv2.COLOR_BGR2GRAY)
    gray_contour_image_resized = cv2.resize(gray_contour_image, (gaussian_heatmaps.shape[2], gaussian_heatmaps.shape[1]))
    masked_heatmaps = np.zeros_like(gaussian_heatmaps)
    max_coords_list = []  # Lista para almacenar las coordenadas de los puntos más altos

    for i in range(len(gaussian_heatmaps)):
        masked_heatmaps[i] = gray_contour_image_resized * gaussian_heatmaps[i]

        # Renormalizar para que los valores estén entre 0 y 1
        max_val = np.max(masked_heatmaps[i])
        if max_val != 0:
            masked_heatmaps[i] = masked_heatmaps[i] / max_val

    return masked_heatmaps

In [34]:
# def apply_max_suppression(landmark_centers, threshold=10, num_landmarks=7):
#     # Sort the landmark centers by confidence or intensity (assuming the second value is confidence)
#     landmark_centers = sorted(landmark_centers, key=lambda x: x[1], reverse=True)

#     while True:
#         # Perform max suppression with the current threshold
#         final_landmarks = [landmark_centers[0]]

#         for center in landmark_centers:
#             if all(np.linalg.norm(np.array(center) - np.array(existing)) > threshold for existing in final_landmarks):
#                 final_landmarks.append(center)

#             if len(final_landmarks) == num_landmarks:
#                 break  # Stop if we reach the desired number of landmarks

#         if len(final_landmarks) == num_landmarks:
#             break  # Break the loop if we have the desired number of landmarks
#         else:
#             threshold -= 1  # Decrease the threshold if the count is less than num_landmarks

#     return final_landmarks

In [35]:
# def save_landmarks_as_masks(landmarks, mask_name, output_directory):
#     mask_output_directory = os.path.join(output_directory, f"{mask_name}")
#     os.makedirs(mask_output_directory, exist_ok=True)
#     for i, landmark in enumerate(landmarks):
#         landmark_image = np.zeros((112, 112, 3), dtype=np.uint8)
#         landmark_path = os.path.join(mask_output_directory, f"{mask_name}_landMark_{i + 1}.png")
#         cv2.circle(landmark_image, tuple(landmark[::-1]), 0, (255, 0, 0), -1)  # Usar radio 0 para dibujar un punto
#         cv2.imwrite(landmark_path, landmark_image)

In [36]:
# def process_directory(directory, output_landmarks_directory, threshold=10):
#     for filename in os.listdir(directory):
#         print(filename)
#         mask_number = int(filename.split("_mask_")[1].split(".")[0])
#         if filename.endswith(f"_mask_{mask_number:04d}.png"):
#             mask_path = os.path.join(directory, filename)
#             mask_name = os.path.splitext(filename)[0]

#             mask = cv2.imread(mask_path)
#             contour_image = extract_contours(mask)
#             heatmaps = generate_heatmaps(img_h, img_w, coords)
#             gaussian_heatmaps = apply_gaussian_filter(heatmaps)

#             masked_heatmaps, max_coords_list = apply_contour_mask(gaussian_heatmaps, contour_image)
#             landmarks = apply_max_suppression(max_coords_list, threshold)
#             save_landmarks_as_masks(landmarks, mask_name, output_landmarks_directory)

In [37]:
def save_heatmaps_as_txt(masked_heatmaps, mask_name, output_directory):
    txt_path = os.path.join(output_directory, f"{mask_name}_heatmaps.txt")
    with open(txt_path, 'w') as f:
        for i, heatmap in enumerate(masked_heatmaps, start=1):
            f.write(f"Landmark {i}:\n")  # Marcar el inicio de una nueva landmark
            for row in heatmap:
                for value in row:
                    f.write(f"{value} ")
                f.write("\n")

def process_directory(directory, output_landmarks_directory, threshold=10):
    for filename in os.listdir(directory):
        mask_number = int(filename.split("_mask_")[1].split(".")[0])
        if filename.endswith(f"_mask_{mask_number:04d}.png"):
            mask_path = os.path.join(directory, filename)
            mask_name = os.path.splitext(filename)[0]

            mask = cv2.imread(mask_path)
            contour_image = extract_contours(mask)
            heatmaps = generate_heatmaps(img_h, img_w, coords)
            gaussian_heatmaps = apply_gaussian_filter(heatmaps)

            masked_heatmaps = apply_contour_mask(gaussian_heatmaps, contour_image)

            # Directorio para los heatmaps
            heatmaps_output_directory = os.path.join(output_landmarks_directory, f"{mask_name}")
            os.makedirs(heatmaps_output_directory, exist_ok=True)

            # Guardar los heatmaps en un archivo txt
            save_heatmaps_as_txt(masked_heatmaps, mask_name, heatmaps_output_directory)

In [38]:
# Directorio raíz donde se guardarán las imágenes de las masks
root_directory = 'D:\\RetoAI'

# Directorio para train
train_directory = os.path.join(root_directory, 'train', 'masks')
# Directorio para test
test_directory = os.path.join(root_directory, 'test', 'masks')
# Directorio para val
val_directory = os.path.join(root_directory, 'val', 'masks')

In [39]:
# Directorio raíz donde se guardarán las imágenes de las masks
root_directory = 'D:\\RetoAI'

# Directorio para train
train_directoryL = os.path.join(root_directory, 'train', 'landMarks')
# Directorio para test
test_directoryL = os.path.join(root_directory, 'test', 'landMarks')
# Directorio para val
val_directoryL = os.path.join(root_directory, 'val', 'landMarks')

# Crear directorios si no existen
for directory in [train_directoryL, test_directoryL, val_directoryL]:
    os.makedirs(directory, exist_ok=True)

In [40]:
for directory in [train_directoryL, test_directoryL, val_directoryL]:
    try:
        os.makedirs(directory, exist_ok=True)
        print(f"Directorio {directory} creado con éxito.")
    except Exception as e:
        print(f"Error al crear el directorio {directory}: {e}")

Directorio D:\RetoAI\train\landMarks creado con éxito.
Directorio D:\RetoAI\test\landMarks creado con éxito.
Directorio D:\RetoAI\val\landMarks creado con éxito.


In [41]:
# Definir las coordenadas y el tamaño de la imagen
img_h, img_w = 112, 112
coords = [(20, 26), (20, 84), (56, 20), (56, 92), (92, 66), (92, 25), (92, 92)]

In [42]:
train_landmarks_df = process_directory(train_directory, train_directoryL)

In [43]:
test_landmarks_df = process_directory(test_directory, test_directoryL)

In [44]:
val_landmarks_df = process_directory(val_directory, val_directoryL)